Number of Teams Each Year

In [0]:
df = spark.read.option("header",True).csv("/FileStore/tables/ipl_commentary_data.csv")

teams_year = df.select("series_name", "team1_name", "team2_name") \
    .withColumnRenamed("team1_name", "team2_name") \
    .union(df.select("series_name","team1_name", "team2_name").withColumnRenamed("team2_name", "team1_name")) \
    .distinct() \
    .groupBy("series_name") \
    .count()

teams_year.show()

+--------------------+-----+
|         series_name|count|
+--------------------+-----+
|Indian Premier Le...|   44|
|Indian Premier Le...|   55|
|Indian Premier Le...|   42|
|Indian Premier Le...|    8|
|                null|    1|
|Indian Premier Le...|   42|
|Indian Premier Le...|   39|
|Indian Premier Le...|   60|
|Indian Premier Le...|   43|
|Indian Premier Le...|   60|
+--------------------+-----+



Number of Matches: Abandoned, Tied, Completed

In [0]:
from pyspark.sql.functions import col

match_status_counts = df.groupBy("match_status").count()

match_status_counts.show()

+------------+------+
|match_status| count|
+------------+------+
|   completed|142630|
|        null|     9|
|        tied|  1975|
+------------+------+



Average Number of ball_no per Match per Inning

In [0]:
avg_balls = df.groupBy("match_id", "over_no") \
    .agg({"ball_no": "count"}) \
    .groupBy("over_no") \
    .avg("count(ball_no)")

avg_balls.show()

+-------+-------------------+
|over_no|avg(count(ball_no))|
+-------+-------------------+
|    8.5|  2.445506692160612|
|    125|                0.0|
|    2.6| 2.3422562141491396|
|    8.2| 2.3288718929254304|
|    7.3| 2.3212237093690247|
|    124|                0.0|
|     51|                0.0|
|      7|                0.0|
|    3.1|  2.369024856596558|
|   16.6| 2.2451737451737452|
|   14.2|  2.390057361376673|
|   17.1|  2.247104247104247|
|    8.3|  2.349904397705545|
|     15|                0.0|
|     54|                0.0|
|    4.2| 2.3365200764818357|
|    9.2| 2.3288718929254304|
|    101|                0.0|
|     11|                0.0|
|   18.1|          2.2265625|
+-------+-------------------+
only showing top 20 rows



Matches Won per Team per Year

In [0]:
matches_won = df.groupBy("series_name", "match_winning_team") \
    .count() \
    .withColumnRenamed("count", "matches_won") \
    .orderBy("series_name", "matches_won", ascending=[True, False])

matches_won.show()

+--------------------+--------------------+-----------+
|         series_name|  match_winning_team|matches_won|
+--------------------+--------------------+-----------+
|                null|                null|          9|
|Indian Premier Le...| Chennai Super Kings|       3643|
|Indian Premier Le...|      Mumbai Indians|       2420|
|Indian Premier Le...|      Delhi Capitals|       2177|
|Indian Premier Le...|     Kings XI Punjab|       1470|
|Indian Premier Le...|Kolkata Knight Ri...|       1430|
|Indian Premier Le...| Sunrisers Hyderabad|       1425|
|Indian Premier Le...|Royal Challengers...|       1235|
|Indian Premier Le...|    Rajasthan Royals|       1212|
|Indian Premier Le...|Match tied (Delhi...|        246|
|Indian Premier Le...|Match tied (Mumba...|        244|
|Indian Premier Le...|      Mumbai Indians|       2600|
|Indian Premier Le...|      Delhi Capitals|       1989|
|Indian Premier Le...| Sunrisers Hyderabad|       1895|
|Indian Premier Le...|Kolkata Knight Ri...|     

Compare collect_list vs collect_set for Batsmen per Team per Match

In [0]:
from pyspark.sql.functions import collect_list, collect_set

df2 = spark.read.option("header", True).csv("/FileStore/tables/ipl_over_data.csv")

batsmen_list_set = df2.groupBy("match_id", "over_batsman1_name") \
    .agg(
        collect_list("over_batsman1_name").alias("batsmen_list"),
        collect_set("over_batsman1_name").alias("batsmen_set")
    )

batsmen_list_set.display(truncate=False)

match_id,over_batsman1_name,batsmen_list,batsmen_set
114960,Ajinkya Rahane,"List(Ajinkya Rahane, Ajinkya Rahane, Ajinkya Rahane, Ajinkya Rahane, Ajinkya Rahane, Ajinkya Rahane)",List(Ajinkya Rahane)
114960,Angkrish Raghuvanshi,"List(Angkrish Raghuvanshi, Angkrish Raghuvanshi)",List(Angkrish Raghuvanshi)
114960,Devdutt Padikkal,"List(Devdutt Padikkal, Devdutt Padikkal)",List(Devdutt Padikkal)
114960,Harshit Rana,List(Harshit Rana),List(Harshit Rana)
114960,Liam Livingstone,List(Liam Livingstone),List(Liam Livingstone)
114960,Philip Salt,"List(Philip Salt, Philip Salt, Philip Salt, Philip Salt, Philip Salt)",List(Philip Salt)
114960,Rajat Patidar,"List(Rajat Patidar, Rajat Patidar)",List(Rajat Patidar)
114960,Ramandeep Singh,"List(Ramandeep Singh, Ramandeep Singh, Ramandeep Singh)",List(Ramandeep Singh)
114960,Rinku Singh,"List(Rinku Singh, Rinku Singh)",List(Rinku Singh)
114960,Spencer Johnson,List(Spencer Johnson),List(Spencer Johnson)
